# Sleep IMU Example

<div class="alert alert-block alert-info">
This example illustrates how to import data from inertial measurement units (IMUs) collected during sleep and compute sleep endpoints.
</div>

<div class="alert alert-block alert-info">
    
<b>Note:</b> An <i>inertial measurement unit (IMU)</i> is a sensor that measures a body's acceleration (using accelerometers) and angular rate (using gyroscopes). In medical and psychological applications IMUs are commonly used for activity monitoring, movement analysis, and many more.
    
</div>

## Setup and Helper Functions

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

from nilspodlib import Dataset

import biopsykit as bp

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
plt.close("all")

palette = bp.colors.fau_palette
sns.set_theme(
    context="notebook", 
    style="ticks", 
    font="sans-serif",
    palette=palette
)

plt.rcParams['figure.figsize'] = (8,4)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

palette

In [ ]:
def display_dict_structure(dict_to_display):
    _display_dict_recursive(dict_to_display)
    
def _display_dict_recursive(dict_to_display):
    if isinstance(dict_to_display, dict):
        display(dict_to_display.keys())
        _display_dict_recursive(list(dict_to_display.values())[0])
    else:
        display("Dataframe shape: {}".format(dict_to_display.shape))
        display(dict_to_display.head())

## Load Dataset

In [ ]:
# Example dataset 
data, fs = bp.example_data.get_sleep_imu_example()

### Alternatively: Load your own dataset
#data, fs = bp.io.nilspod.load_dataset_nilspod("<path-to-imu-file>") # or any other way to load a file containing IMU data

In [ ]:
data.head()

## Plot IMU Data

In [ ]:
fig, ax = bp.sleep.plotting.sleep_imu_plot(data, downsample_factor=5*fs)

## Compute Sleep Endpoints

The Sleep Processing Pipeline (which is performed in `biopsykit.sleep.sleep_processing_pipeline.predict_pipeline_acceleration()`) consists of the following steps:

1. **Activity Count Conversion**: Convert (3-axis) raw acceleration data into activity counts. Most sleep/wake detection algorithms use activity counts (as typically provided by Actigraphs) as input data.  
1. **Wear Detection**: Detect wear and non-wear periods. Afterwards, cut data to longest continuous wear block.  
1. **Rest Periods**: Detect rest periods, i.e., periods with large physical inactivity. The longest continuous rest period (*Major Rest Period*) is used to determine the *Bed Interval*, i.e., the period spent in bed.  
1. **Sleep/Wake Detection**: Apply sleep/wake detection to classify phases of sleep and wake.  
1. **Sleep Endpoint Computation**: Compute Sleep Endpoints from sleep/wake detection results and bed interval. The following sleep endpoints are supported:
    * `date`: Date of the sleep recording.  
      **Note**: By convention, this field will return the *previous* day if the Bed Interval started *after* 12 a.m., i.e. the subject went to bed after midnight. By this convention, one always night always spans *two days*.
    * `sleep_onset`: Sleep Onset, i.e., time of falling asleep (when *first* sleep phase started), in absolute time
    * `wake_onset`: Wake Onset, i.e., time of awakening (*last* sleep phase ended), in absolute time
    * `total_sleep_duration`: Total duration spent sleeping, i.e., the duration between *Sleep Onset* and *Wake Onset*, in minutes
    * `net_sleep_duration`: Net duration spent sleeping, in minutes
    * `bed_interval_start`: Bed Interval Start, i.e., time when subject went to bed, in absolute time
    * `bed_interval_end`: Bed Interval End, i.e., time when subject left bed, in absolute time
    * `sleep_efficiency`: Ratio of `net_sleep_duration`, i.e., the time actually spent sleeping and `total_sleep_duration`, i.e., the total time between *Sleep Onset* and *Wake Onset*, in percent
    * `sleep_onset_latency`: Sleep Onset Latency, i.e., time in bed needed to fall asleep (difference between *Sleep Onset* and *Bed Interval Start*), in minutes
    * `getup_latency`: Getup Latency, i.e., time in bed after awakening until getting up (difference between *Bed Interval End* and *Wake Onset*), in minutes
    * `wake_after_sleep_onset`: Wake After Sleep Onset (WASO), i.e., total duration awake after falling asleep (after *Sleep Onset* and before *Wake Onset*), in minutes
    * `sleep_bouts`: List with start and end times of sleep bouts
    * `wake_bouts`: List with start and end times of wake bouts
    * `number_wake_bouts`: Total number of wake bouts
    

The results are stored in a dictionary:

In [ ]:
sleep_results = bp.sleep.sleep_processing_pipeline.predict_pipeline_acceleration(data, sampling_rate=fs, sleep_wake_scale_factor=0.1)
sleep_endpoints = sleep_results['sleep_endpoints']

In [ ]:
sleep_endpoints.keys()

## Plot IMU Data with Sleep Endpoints

### Cut Data to Wear Period

For nicer visualization the IMU data are cut to the longest wear period before plotting:

In [ ]:
wear_detection = bp.signals.imu.wear_detection.WearDetection(sampling_rate=fs)

data = wear_detection.cut_to_wear_block(data, sleep_results['major_wear_block'])

In [ ]:
df = bp.sleep.sleep_endpoints.endpoints_as_df(sleep_endpoints)
df

### Plot Sleep Data

In [ ]:
fig, ax = bp.sleep.plotting.sleep_imu_plot(data, sleep_endpoints=sleep_endpoints, downsample_factor=5*fs)

### Export Sleep Endpoints

In [ ]:
#bp.io.sleep.save_sleep_endpoints("../example_data/sleep_endpoints.csv", df)